ETL using BSE api and loading to mysql  


In [1]:
import pandas as pd
from bsedata.bse import BSE 
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

excel_stocks = "https://github.com/suhascp7/bse_py/raw/refs/heads/master/Equity.xlsx"
engine_excel = 'openpyxl'
stockDf = pd.read_excel(excel_stocks,engine=engine_excel)
stockDf.head(2)

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas


Getting nifty50, from the whole market 

In [3]:
# listOf_Nifty50_Stocks = [ 
#     "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
#     "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
#     "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
#     "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
#     "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
#     "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
#     "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
#     "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
#     "UPL", "ULTRACEMCO", "WIPRO"
# ]

listOf_Nifty50_Stocks = [ 
    "ADANIENT", "ADANIPORTS", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
     "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", 
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM"
   
]



stockDf['Security Code']= stockDf['Security Code'].astype(str)
nifty50 = stockDf[stockDf['Security Id'].isin(listOf_Nifty50_Stocks)].reset_index(drop=True)
nifty50.columns = nifty50.columns.str.replace(' ', '')


In [4]:
bseObject = BSE(update_codes=True) 

In [5]:
listof_stocksDict= []
seqCode = nifty50['SecurityCode']
for code in seqCode:
    try:
        dictStock = bseObject.getQuote(code)
        dictStock.pop('sell')
        dictStock.pop('buy')
        listof_stocksDict.append(dictStock)
        time.sleep(0.5)
    except IndexError:
        print(f'IndexError for {code}, data not available')


nifty50Today = pd.DataFrame(listof_stocksDict)

In [6]:
nifty50Today.head(2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,group,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,52weekHigh,52weekLow,weightedAvgPrice,totalTradedValue,totalTradedQuantity,2WeekAvgQuantity,marketCapFull,marketCapFreeFloat
0,Bajaj Finance Limited,6552.90,-11.90,-0.18,14 Nov 24 | 04:00 PM,BAJFINANCE,500034,A / BSE SENSEX,2.00,Financial Services,6564.80,6570.00,6642.45,6532.00,7829.95,6190.00,6568.51,11.49 Cr.,0.17 Lakh,0.37 Lakh,"4,05,622.10 Cr.","1,82,529.94 Cr."
1,CIPLA LTD.,1500.15,-5.10,-0.34,14 Nov 24 | 04:00 PM,CIPLA,500087,A / BSE 100,2.00,Healthcare,1505.25,1499.95,1515.25,1491.45,1702.00,1165.10,1502.52,3.69 Cr.,0.25 Lakh,0.59 Lakh,"1,21,151.43 Cr.","83,594.49 Cr."


In [ ]:
nifty50Today.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50Today.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50Today.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50Today.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)

# Converting 'updatedOn'  to datetime and extract date
nifty50Today['updatedOn'] = pd.to_datetime(nifty50Today['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

# Check if there are any invalid or missing date values
if pd.isna(nifty50Today['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50Today['updatedOn'] = pd.to_datetime(nifty50Today['updatedOn'].dt.date)

if 'totalTradedValueCr' not in nifty50Today.columns:
   # Assuming nifty50Today is your DataFrame
    nifty50Today['totalTradedValueCr'] = pd.to_numeric(nifty50Today['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50Today['totalTradedQuantityLakh'] = pd.to_numeric(nifty50Today['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50Today['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50Today['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50Today['marketCapFullCr'] = pd.to_numeric(nifty50Today['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50Today['marketCapFreeFloatCr'] = pd.to_numeric(nifty50Today['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'

    # Drop original columns
    nifty50Today.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1, inplace=True)

nifty50Today.head(n=2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,6552.90,-11.90,-0.18,2024-11-14,BAJFINANCE,500034,A / BSE SENSEX,2.00,Financial Services,6564.80,6570.00,6642.45,6532.00,7829.95,6190.00,6568.51,11.49,0.17,0.37,405622.10,182529.94
1,CIPLA LTD.,1500.15,-5.10,-0.34,2024-11-14,CIPLA,500087,A / BSE 100,2.00,Healthcare,1505.25,1499.95,1515.25,1491.45,1702.00,1165.10,1502.52,3.69,0.25,0.59,121151.43,83594.49


In [ ]:
import mysql.connector

    